In [18]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from FoodSeg103_data_handler import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create scribbles

Load the ground truths as batch

In [19]:
img_nums = [n for n in range(0, 4900, 130)] #[1382] #2750 #1234 #2314
gts = load_food_batch(img_nums, load_images=False)[1]
num_imgs = len(gts)
print(f"Total images: {num_imgs}")

Total images: 38


Check and filter for resolution

In [20]:
resol = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}

# Check for images with resolution over a certain threshold
thresh = 250_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
   0:   196,608 pixels
 130:   126,280 pixels
 260:    65,536 pixels
 390:    65,536 pixels
 520: 7,990,272 pixels (!)
 650:   196,608 pixels
 780:    65,536 pixels
 910:   201,850 pixels
1040: 7,990,272 pixels (!)
1170:    65,536 pixels
1300: 12,780,288 pixels (!)
1430:   110,600 pixels
1560: 1,024,000 pixels (!)
1690:    65,536 pixels
1820:    65,536 pixels
1950:   196,608 pixels
2080:    65,536 pixels
2210: 7,990,272 pixels (!)
2340: 1,304,164 pixels (!)
2470:    65,536 pixels
2600:   614,400 pixels (!)
2730:   196,608 pixels
2860:   196,608 pixels
2990:   196,608 pixels
3120:   196,608 pixels
3250:   196,608 pixels
3380:   207,872 pixels
3510:    65,536 pixels
3640: 2,795,520 pixels (!)
3770:   195,584 pixels
3900:    65,536 pixels
4030:    65,536 pixels
4160:   196,608 pixels
4290:    65,536 pixels
4420:   750,000 pixels (!)
4550:   812,844 pixels (!)
4680:   196,608 pixels
4810:   111,930 pixels


In [21]:
# Filter out those images
gts = {im_num: gt for im_num, gt in gts.items() if resol[im_num] <= thresh}
num_imgs = len(gts)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 250,000: 28
   0: 196,608 pixels
 130: 126,280 pixels
 260:  65,536 pixels
 390:  65,536 pixels
 650: 196,608 pixels
 780:  65,536 pixels
 910: 201,850 pixels
1170:  65,536 pixels
1430: 110,600 pixels
1690:  65,536 pixels
1820:  65,536 pixels
1950: 196,608 pixels
2080:  65,536 pixels
2470:  65,536 pixels
2730: 196,608 pixels
2860: 196,608 pixels
2990: 196,608 pixels
3120: 196,608 pixels
3250: 196,608 pixels
3380: 207,872 pixels
3510:  65,536 pixels
3770: 195,584 pixels
3900:  65,536 pixels
4030:  65,536 pixels
4160: 196,608 pixels
4290:  65,536 pixels
4680: 196,608 pixels
4810: 111,930 pixels


Define scribbles parameters

In [22]:
# Which scribbles to create
mode = "all"
class_dist = "balanced"
enforce_max_perc = True
# bin_width = [(0.05, 1), (0.1, 1),
            #  (0.25, 2), (0.5, 2), (1, 2),
            #  (2.5, 3), (5, 3),
            #  (10, 4)]
bins = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1, 2.5, 5, 10] #[0.01, 0.03, 0.1, 0.3, 1, 3, 10]#[0.05, 0.25, 1, 5]
margin = 0.75
scribble_width = 2
# suff = ["TEST"] #["w3"] #standing for width 3
s = "TEST"
rel_scribble_len = 10
scribbles_seed = 1

# Where to save the scribbles
folder_path = "./FoodSeg103_results/data/food_run02"

# What to do
save_res = False
show_res = False
print_steps = True

Loop and create scribbles

In [23]:
percentages = {}
for bin in bins:
# for bin, scribble_width in bin_width:
    percentages[bin] = []
    for img_num, gt in gts.items():
        print(f"IMG {img_num}: bin {bin}, suff {s}")
        np.random.seed(scribbles_seed)
        scribbles, perc_labelled = create_food_scribble(gt, folder_path, img_num, bin=bin, margin=margin, rel_scribble_len=rel_scribble_len, scribble_width=scribble_width, mode=mode,
                                                        class_dist=class_dist, enforce_max_perc=enforce_max_perc,
                                                        save_res=save_res, suff=s, show_res=show_res, image=None, print_steps=print_steps)
        percentages[bin].append(perc_labelled)
        print("\n")

IMG 0: bin 10, suff TEST

max. perc.: 10, margin: 0.75, rel_scribble_len: 10.00, width: 2, mode: all, class_dist: balanced, enforce_max_perc: True, print_steps: True

CLASS 1, max. pixel: 10.199% = 4821 pixels
   sk_max_pix: 1607.00, sq_size: 44, sq_pix_range: (44, 176)
      prim_sk_squares pix: 1591 = 3.366%
      sec_sk_squares pix: 1565 = 3.311%
   lines_max_pix: 1607.00, line_pix_range: (44, 176)
      lines pix: 1582 = 3.347%
CLASS 1 pixels: 4658 = 9.854% 

CLASS 67, max. pixel: 8.356% = 6118 pixels
   sk_max_pix: 2039.00, sq_size: 44, sq_pix_range: (44, 176)
      NOTE: All pixels in the skeleton were added (2033).
      prim_sk_squares pix: 2033 = 2.776%
      sec_sk_squares pix: 2034 = 2.778%
   lines_max_pix: 2039.00, line_pix_range: (44, 176)
      lines pix: 1998 = 2.729%
CLASS 67 pixels: 5856 = 7.997% 

CLASS 91, max. pixel: 13.446% = 3912 pixels
   sk_max_pix: 1304.00, sq_size: 44, sq_pix_range: (44, 176)
      prim_sk_squares pix: 1276 = 4.385%
      sec_sk_squares pix: 

KeyboardInterrupt: 

Quick report of labelling percentages:

In [ ]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

Check percentages of saved scribbles

In [ ]:
perc_dict = {}
for bin in bins:
    bin_perc_list = []
    for img_num in range(0,540,20):
        scrib = np.array(Image.open(f"{folder_path}{img_num:03d}_scribbles_{mode}_{bin_for_file(bin)}_{s}.png"))
        perc_labelled = np.sum(scrib > 0) / scrib.size
        bin_perc_list.append(perc_labelled)
    perc_dict[bin] = bin_perc_list

Show the last created scribble with its own ground truth, and (optionally) all other ground truths

In [ ]:
v = napari.Viewer()
v.add_labels(gt, name="Ground Truth")
v.add_labels(scribbles, name="Scribbles")

In [ ]:
for g in gts:
    v.add_labels(gts[g], name=f"GT {g}")